## Set up
Model selection based on https://christophm.github.io/interpretable-ml-book/simple.html and availability in sklearn

In [ ]:
import os
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score, recall_score
import numpy as np

In [ ]:
# Build full cx df
columns = ['Closed comedo', 'Dermatoglyph disruption', 'Open comedo', 'Papule', 'Patch', 'Plaque', 'Pustule', 'Scale', 'Scar', 'Sun damage']
cx_pairs = []
for mask in os.listdir("/home/ubuntu/store/dermx_cleaned_masks/fusion_masks"):
    _, _, image_id, cx = mask.split('.')[0].split('_')
    cx_pairs.append((image_id + '.jpeg', cx))
    
image_ids, cxs = zip(*cx_pairs)
cx_df = pd.Series(cxs, image_ids)
cx_df.index.name = 'image_id'
cx_df = pd.get_dummies(cx_df).groupby(by='image_id').max()[columns]
cx_df

,Closed comedo,Dermatoglyph disruption,Open comedo,Papule,Patch,Plaque,Pustule,Scale,Scar,Sun damage
image_id,,,,,,,,,,
000498HB.jpeg,0,0,0,0,1,0,0,1,0,1
005102HB.jpeg,0,0,0,1,1,1,0,1,0,1
009001HB.jpeg,0,0,0,1,0,0,1,0,0,0
016003HB.jpeg,0,0,0,1,1,1,0,1,0,0
016013HB.jpeg,0,0,0,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...
wart4--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg,0,0,0,1,0,1,0,0,0,0
wart5--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg,0,1,0,1,0,1,0,0,0,0
wart6--WatermarkedWyJXYXRlcm1hcmtlZCJd.jpeg,0,1,0,1,0,1,0,0,0,0


## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
fold_recalls = []
fold_f1s = []
fold_accuracies = []
fold_specificities = []

for fold_idx in range(10):
    clf = LogisticRegression()
    train_df = pd.read_csv(f'/home/ubuntu/store/dermx-folds/fold_{fold_idx}/metadata_fold_{fold_idx}_train.csv')
    test_df = pd.read_csv(f'/home/ubuntu/store/dermx-folds/fold_{fold_idx}/metadata_fold_{fold_idx}_test.csv')
    
    x_train = cx_df.loc[train_df['current_filename']]
    y_train = train_df['diagnosis']
    x_test = cx_df.loc[test_df['current_filename']]
    y_test = test_df['diagnosis']
    clf.fit(x_train, y_train)
    y_hat_test = clf.predict(x_test)
    
    report = classification_report(y_test, y_hat_test, output_dict=True)
    fold_recalls.append(report['macro avg']['recall'])
    fold_f1s.append(report['macro avg']['f1-score'])
    fold_accuracies.append(accuracy_score(y_test, y_hat_test))
    fold_specificities.append(recall_score(y_test, y_hat_test, pos_label=0, average='macro'))

/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note

In [ ]:
print(f'{np.round(np.array(fold_recalls).mean(), 2)} \pm {np.round(np.array(fold_recalls).std(), 2)}')

0.87 \pm 0.04


In [ ]:
print(f'{np.round(np.array(fold_f1s).mean(), 2)} \pm {np.round(np.array(fold_f1s).std(), 2)}')

0.86 \pm 0.04


In [ ]:
print(f'{np.round(np.array(fold_specificities).mean(), 2)} \pm {np.round(np.array(fold_specificities).std(), 2)}')

0.87 \pm 0.04


In [ ]:
print(f'{np.round(np.array(fold_accuracies).mean(), 2)} \pm {np.round(np.array(fold_accuracies).std(), 2)}')

0.86 \pm 0.04


## Decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
fold_recalls = []
fold_f1s = []
fold_accuracies = []
fold_specificities = []

for fold_idx in range(10):
    clf = DecisionTreeClassifier()
    train_df = pd.read_csv(f'/home/ubuntu/store/dermx-folds/fold_{fold_idx}/metadata_fold_{fold_idx}_train.csv')
    test_df = pd.read_csv(f'/home/ubuntu/store/dermx-folds/fold_{fold_idx}/metadata_fold_{fold_idx}_test.csv')
    
    x_train = cx_df.loc[train_df['current_filename']]
    y_train = pd.get_dummies(train_df['diagnosis'])
    x_test = cx_df.loc[test_df['current_filename']]
    y_test = pd.get_dummies(test_df['diagnosis'])
    clf.fit(x_train, y_train)
    y_hat_test = clf.predict(x_test)
    
    report = classification_report(y_test, y_hat_test, output_dict=True)
    fold_recalls.append(report['macro avg']['recall'])
    fold_f1s.append(report['macro avg']['f1-score'])
    fold_accuracies.append(accuracy_score(y_test, y_hat_test))
    fold_specificities.append(recall_score(y_test, y_hat_test, pos_label=0, average='macro'))

/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/venv/pytorch/lib/python3.6/site-package

In [ ]:
print(f'{np.round(np.array(fold_recalls).mean(), 2)} \pm {np.round(np.array(fold_recalls).std(), 2)}')

0.84 \pm 0.05


In [ ]:
print(f'{np.round(np.array(fold_f1s).mean(), 2)} \pm {np.round(np.array(fold_f1s).std(), 2)}')

0.85 \pm 0.05


In [ ]:
print(f'{np.round(np.array(fold_specificities).mean(), 2)} \pm {np.round(np.array(fold_specificities).std(), 2)}')

0.84 \pm 0.05


In [ ]:
print(f'{np.round(np.array(fold_accuracies).mean(), 2)} \pm {np.round(np.array(fold_accuracies).std(), 2)}')

0.84 \pm 0.05


## kNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
fold_recalls = []
fold_f1s = []
fold_accuracies = []
fold_specificities = []

for fold_idx in range(10):
    clf = KNeighborsClassifier()
    train_df = pd.read_csv(f'/home/ubuntu/store/dermx-folds/fold_{fold_idx}/metadata_fold_{fold_idx}_train.csv')
    test_df = pd.read_csv(f'/home/ubuntu/store/dermx-folds/fold_{fold_idx}/metadata_fold_{fold_idx}_test.csv')
    
    x_train = cx_df.loc[train_df['current_filename']]
    y_train = pd.get_dummies(train_df['diagnosis'])
    x_test = cx_df.loc[test_df['current_filename']]
    y_test = pd.get_dummies(test_df['diagnosis'])
    clf.fit(x_train, y_train)
    y_hat_test = clf.predict(x_test)
    
    report = classification_report(y_test, y_hat_test, output_dict=True)
    fold_recalls.append(report['macro avg']['recall'])
    fold_f1s.append(report['macro avg']['f1-score'])
    fold_accuracies.append(accuracy_score(y_test, y_hat_test))
    fold_specificities.append(recall_score(y_test, y_hat_test, pos_label=0, average='macro'))

/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classificat

In [ ]:
print(f'{np.round(np.array(fold_recalls).mean(), 2)} \pm {np.round(np.array(fold_recalls).std(), 2)}')

0.8 \pm 0.04


In [ ]:
print(f'{np.round(np.array(fold_f1s).mean(), 2)} \pm {np.round(np.array(fold_f1s).std(), 2)}')

0.8 \pm 0.05


In [ ]:
print(f'{np.round(np.array(fold_specificities).mean(), 2)} \pm {np.round(np.array(fold_specificities).std(), 2)}')

0.8 \pm 0.04


In [ ]:
print(f'{np.round(np.array(fold_accuracies).mean(), 2)} \pm {np.round(np.array(fold_accuracies).std(), 2)}')

0.8 \pm 0.05


## Naive Bayes

In [ ]:
from sklearn.naive_bayes import CategoricalNB

In [ ]:
fold_recalls = []
fold_f1s = []
fold_accuracies = []
fold_specificities = []

for fold_idx in range(10):
    clf = CategoricalNB()
    train_df = pd.read_csv(f'/home/ubuntu/store/dermx-folds/fold_{fold_idx}/metadata_fold_{fold_idx}_train.csv')
    test_df = pd.read_csv(f'/home/ubuntu/store/dermx-folds/fold_{fold_idx}/metadata_fold_{fold_idx}_test.csv')
    
    x_train = cx_df.loc[train_df['current_filename']]
    y_train = train_df['diagnosis']
    x_test = cx_df.loc[test_df['current_filename']]
    y_test = test_df['diagnosis']
    clf.fit(x_train, y_train)
    y_hat_test = clf.predict(x_test)
    
    report = classification_report(y_test, y_hat_test, output_dict=True)
    fold_recalls.append(report['macro avg']['recall'])
    fold_f1s.append(report['macro avg']['f1-score'])
    fold_accuracies.append(accuracy_score(y_test, y_hat_test))
    fold_specificities.append(recall_score(y_test, y_hat_test, pos_label=0, average='macro'))

/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note that pos_label (set to 0) is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/home/ubuntu/venv/pytorch/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1301: UserWarning: Note

In [ ]:
print(f'{np.round(np.array(fold_recalls).mean(), 2)} \pm {np.round(np.array(fold_recalls).std(), 2)}')

0.86 \pm 0.05


In [ ]:
print(f'{np.round(np.array(fold_f1s).mean(), 2)} \pm {np.round(np.array(fold_f1s).std(), 2)}')

0.86 \pm 0.05


In [ ]:
print(f'{np.round(np.array(fold_specificities).mean(), 2)} \pm {np.round(np.array(fold_specificities).std(), 2)}')

0.86 \pm 0.05


In [ ]:
print(f'{np.round(np.array(fold_accuracies).mean(), 2)} \pm {np.round(np.array(fold_accuracies).std(), 2)}')

0.86 \pm 0.05
